In [1]:
from GROFRAME import GROFRAME

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [38]:
import numpy as np
import pandas as pd

class GRO:
    import numpy as np
    import pandas as pd
    def __init__(self, fname):
        self.fname = fname
        self.all_lines = open(self.fname, encoding = "utf-8").readlines()
        length = len(self.all_lines)
        if int(self.all_lines[1]) != (length - 3):
            print("Number of atoms mismatch!!")
        self.dimension = np.array(self.all_lines[-1].split(), dtype = float)
    def make_line(self,i):
        Line = self.all_lines[i]
        mLine = self.getLine(Line)
        mDict = {"rid" : mLine[0], "resname" : mLine[1], "name" : mLine[2],
                 "id" : mLine[3], "x" : mLine[4], "y" : mLine[5], "z" : mLine[6]}
        return mDict
    def getLine(self,Line):
        rid = Line[:5].strip()
        resname = Line[5:10].strip()
        nname = Line[10:15].strip()
        nid = Line[15:20].strip()
        x = Line[20:28].strip()
        y = Line[28:36].strip()
        z = Line[36:44].strip()
        return rid, resname, nname, nid, x, y, z
    def makeDataFrame(self):
        totalAtomNumber = int(self.all_lines[1])
        #print(totalAtomNumber)
        #df = pd.DataFrame({"rid" : 0, "resname" : 1, "name" : 2,
        #         "id" : 3, "x" : 4, "y" : 5, "z" : 6}, index = [0])
        Lines = self.all_lines[2:-1]
        rid = np.array([self.getLine(Lines[i])[0] for i in range(totalAtomNumber)], dtype = int)
        resname = np.array([self.getLine(Lines[i])[1] for i in range(totalAtomNumber)], dtype = str)
        name = np.array([self.getLine(Lines[i])[2] for i in range(totalAtomNumber)], dtype = str)
        nid = np.array([self.getLine(Lines[i])[3] for i in range(totalAtomNumber)], dtype = int)
        x = np.array([self.getLine(Lines[i])[4] for i in range(totalAtomNumber)], dtype = float)
        y = np.array([self.getLine(Lines[i])[5] for i in range(totalAtomNumber)], dtype = float)
        z = np.array([self.getLine(Lines[i])[6] for i in range(totalAtomNumber)],dtype = float)
        df = pd.DataFrame({"rid" : rid, "resname" : resname, "name" : name,
                 "id" :nid , "x" : x, "y" : y, "z" : z})
        return df
    def oneLine(self, rid, resname, nname, nid, x, y, z):
        a = str(rid).rjust(5)
        b = str(resname).ljust(5)
        c = str(nname).rjust(5)
        d = str(nid).rjust(5)
        x = str(x).rjust(8)
        y = str(y).rjust(8)
        z = str(z).rjust(8)
        return a+b+c+d+x+y+z
    
    def df2gro(self, clean_df, fname = "Some_file.gro"):
        clean_file = open(fname, "w")
        clean_file.write("Cleaned using groframe! \n")
        clean_file.write(f" {len(clean_df)}\n")
        for i in range(len(clean_df)):
            string = self.oneLine(*clean_df.iloc[i])
            clean_file.write(string + "\n")
        clean_file.write(self.all_lines[-1] + "\n")
        clean_file.close()
        return "Success!"


In [39]:
frame3 = GRO("Sol_Box_3.gro")

In [40]:
frame2 = GRO("Sol_Box_2.gro")
frame1 = GRO("Sol_Box_1.gro")

In [41]:
df3 = frame3.makeDataFrame()
df2 = frame2.makeDataFrame()
df1 = frame1.makeDataFrame()

In [48]:
df1[df1.resname == "ICE"].z.min(), df1[df1.resname == "ICE"].z.max() 

(4.811, 9.641)

In [50]:
rids1 = df1[df1.resname=="SOL"][((df1[df1.resname == "SOL"].z > 4.811).values * (df1[df1.resname == "SOL"].z < 9.641).values)].rid.unique()
rids2 = df2[df2.resname=="SOL"][((df2[df2.resname == "SOL"].z > 4.811).values * (df2[df2.resname == "SOL"].z < 9.641).values)].rid.unique()
rids3 = df3[df3.resname=="SOL"][((df3[df3.resname == "SOL"].z > 4.811).values * (df3[df3.resname == "SOL"].z < 9.641).values)].rid.unique()

In [51]:
clean_df1 = df1[~df1.rid.isin(rids1)]
clean_df2 = df2[~df2.rid.isin(rids2)]
clean_df3 = df3[~df3.rid.isin(rids3)]

In [55]:
frame3.df2gro(clean_df1, fname = "Cleaned_Box_3.gro")
frame2.df2gro(clean_df2, fname = "Cleaned_Box_2.gro")
frame1.df2gro(clean_df3, fname = "Cleaned_Box_1.gro")

'Success!'

In [54]:
clean_df3

,rid,resname,name,id,x,y,z
0,1,ICE,OW,1,1.142,0.460,4.883
1,1,ICE,HW1,2,1.238,0.459,4.884
2,1,ICE,HW2,3,1.116,0.407,4.958
3,1,ICE,MW,4,1.151,0.453,4.893
4,2,ICE,OW,5,0.455,0.089,5.022
...,...,...,...,...,...,...,...
40947,10996,SOL,MW,40948,4.043,3.864,13.248
40948,10997,SOL,OW,40949,3.803,3.961,13.111
40949,10997,SOL,HW1,40950,3.770,4.040,13.155
40950,10997,SOL,HW2,40951,3.858,3.994,13.040
